In [2]:
import pandas as pd
import numpy as np
import pickle

# Unsupervised Learning
from sklearn import cluster

# Game Libraries
import random
import time

## Cleaning

In [3]:
nba = pd.read_excel('New_Dataset.xlsx', 'Folha2')
nba

,Ranking,Player,Team,GP,MPG,FGM,FGA,FG%,3PM,3PA,...,TOV,PF,ORB,DRB,RPG,APG,SPG,BPG,PPG,Season
0,1,Kevin Durant,BRK,23,36.2,10.3,19.6,0.529,1.7,4.6,...,3.0,1.6,0.5,7.2,7.7,5.4,0.7,0.7,28.4,2021
1,2,Stephen Curry,GSW,23,34.2,9.0,20.3,0.440,5.5,13.2,...,3.1,1.8,0.6,5.0,5.6,6.5,1.8,0.5,27.7,2021
2,3,Giannis Antetokounmpo,MIL,22,32.4,9.8,18.5,0.530,1.2,4.1,...,3.2,3.0,1.9,9.9,11.8,5.8,1.0,1.7,27.6,2021
3,4,DeMar DeRozan,CHI,24,35.3,9.5,19.0,0.498,0.8,2.5,...,2.0,2.3,0.6,4.7,5.3,4.1,0.9,0.4,26.4,2021
4,5,Trae Young,ATL,25,34.2,9.1,20.0,0.456,2.8,7.0,...,4.0,1.7,0.7,3.2,4.0,9.4,0.9,0.1,26.3,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,270,Rodney McGruder,LAC,56,15.6,1.3,3.2,0.398,0.4,1.6,...,0.4,1.3,0.5,2.2,2.7,0.6,0.5,0.1,3.3,2019
834,271,Goga Bitadze,IND,54,8.7,1.3,2.8,0.467,0.1,0.8,...,0.5,1.2,0.5,1.5,2.0,0.4,0.2,0.7,3.2,2019
835,272,Matt Dellavedova,CLE,57,14.4,1.1,3.1,0.354,0.4,1.6,...,1.0,1.4,0.3,1.0,1.3,3.2,0.4,0.0,3.1,2019
836,273,Terance Mann,LAC,41,8.8,0.9,1.9,0.468,0.2,0.5,...,0.4,1.1,0.2,1.1,1.3,1.3,0.3,0.1,2.4,2019


In [4]:
nba = nba.drop(['Ranking', 'Team', 'GP'], axis=1)
nba

,Player,MPG,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,...,TOV,PF,ORB,DRB,RPG,APG,SPG,BPG,PPG,Season
0,Kevin Durant,36.2,10.3,19.6,0.529,1.7,4.6,0.377,6.0,6.8,...,3.0,1.6,0.5,7.2,7.7,5.4,0.7,0.7,28.4,2021
1,Stephen Curry,34.2,9.0,20.3,0.440,5.5,13.2,0.416,4.3,4.6,...,3.1,1.8,0.6,5.0,5.6,6.5,1.8,0.5,27.7,2021
2,Giannis Antetokounmpo,32.4,9.8,18.5,0.530,1.2,4.1,0.300,6.8,10.0,...,3.2,3.0,1.9,9.9,11.8,5.8,1.0,1.7,27.6,2021
3,DeMar DeRozan,35.3,9.5,19.0,0.498,0.8,2.5,0.333,6.6,7.5,...,2.0,2.3,0.6,4.7,5.3,4.1,0.9,0.4,26.4,2021
4,Trae Young,34.2,9.1,20.0,0.456,2.8,7.0,0.392,5.3,6.0,...,4.0,1.7,0.7,3.2,4.0,9.4,0.9,0.1,26.3,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,Rodney McGruder,15.6,1.3,3.2,0.398,0.4,1.6,0.270,0.3,0.6,...,0.4,1.3,0.5,2.2,2.7,0.6,0.5,0.1,3.3,2019
834,Goga Bitadze,8.7,1.3,2.8,0.467,0.1,0.8,0.190,0.4,0.6,...,0.5,1.2,0.5,1.5,2.0,0.4,0.2,0.7,3.2,2019
835,Matt Dellavedova,14.4,1.1,3.1,0.354,0.4,1.6,0.231,0.6,0.6,...,1.0,1.4,0.3,1.0,1.3,3.2,0.4,0.0,3.1,2019
836,Terance Mann,8.8,0.9,1.9,0.468,0.2,0.5,0.350,0.4,0.7,...,0.4,1.1,0.2,1.1,1.3,1.3,0.3,0.1,2.4,2019


In [5]:
nba = nba.groupby(['Player']).mean()

## K-Means - Clusters

In [6]:
kmeans = cluster.KMeans(n_clusters=5)
kmeans.fit(nba.drop(['Season'], axis=1))
pred = kmeans.predict(nba.drop(['Season'], axis=1))

In [7]:
nba["cluster"] = pred

In [8]:
nba['cluster'].value_counts()

0    120
3     95
1     87
2     69
4     44
Name: cluster, dtype: int64

In [48]:
WKP_OVRL = nba[nba.cluster ==2].reset_index() # Weakest Players - Overall
# WKP_OVRL.sort_values(by='FGM', ascending=True)
# WKP_OVRL

In [57]:
BP_OVRL = nba[nba.cluster ==4].reset_index() # Best Players - Overall
# BP_OVRL.sort_values(by='FGM', ascending=True)
BP_OVRL

,index,Player,MPG,FGM,FGA,FG%,3PM,3PA,3P%,FTM,...,PF,ORB,DRB,RPG,APG,SPG,BPG,PPG,Season,cluster
0,13,Andrew Wiggins,32.666667,7.366667,15.733333,0.470667,2.100000,5.533333,0.376000,2.866667,...,2.066667,1.200000,3.600000,4.800000,2.600000,0.833333,0.833333,19.700000,2020.0,4
1,15,Anthony Davis,35.050000,9.250000,18.000000,0.513500,0.800000,2.750000,0.259000,5.850000,...,2.400000,2.650000,7.100000,9.750000,3.150000,1.400000,2.250000,25.100000,2020.0,4
2,16,Anthony Edwards,34.300000,7.500000,17.900000,0.420000,2.650000,7.850000,0.335000,2.900000,...,2.000000,1.050000,4.300000,5.400000,3.300000,1.350000,0.650000,20.600000,2020.5,4
3,33,Bradley Beal,35.833333,10.033333,21.800000,0.459000,2.233333,6.800000,0.322667,5.800000,...,2.200000,1.000000,3.533333,4.533333,5.466667,1.066667,0.366667,28.100000,2020.0,4
4,35,Brandon Ingram,34.300000,8.366667,18.300000,0.457667,2.166667,5.733333,0.379333,4.566667,...,2.333333,0.766667,4.966667,5.733333,4.600000,0.700000,0.533333,23.500000,2020.0,4
5,41,C.J. McCollum,35.266667,8.433333,19.000000,0.444333,3.200000,8.133333,0.391333,1.933333,...,2.200000,0.800000,3.300000,4.066667,4.533333,0.933333,0.566667,21.966667,2020.0,4
6,59,Cole Anthony,34.000000,6.800000,16.300000,0.421000,2.600000,7.500000,0.345000,3.400000,...,2.500000,0.500000,5.800000,6.300000,5.700000,0.900000,0.200000,19.600000,2021.0,4
7,60,Collin Sexton,32.366667,7.633333,16.266667,0.465667,1.366667,4.133333,0.331667,3.766667,...,2.366667,1.033333,2.133333,3.166667,3.166667,0.966667,0.100000,20.366667,2020.0,4
8,63,D'Angelo Russell,32.700000,6.600000,17.200000,0.382000,2.900000,8.900000,0.323000,3.000000,...,2.200000,0.500000,3.400000,3.900000,6.400000,0.900000,0.800000,19.000000,2021.0,4
9,67,Damian Lillard,36.333333,8.600000,19.566667,0.437000,3.666667,9.933333,0.364667,5.933333,...,1.533333,0.466667,3.666667,4.166667,7.766667,0.900000,0.333333,26.733333,2020.0,4


In [50]:
# BP_OVRL[BP_OVRL['Player'].isin(['Stephen Curry', 'Tobias Harris'])]['3PM'].values.sum()

In [51]:
AST_OVRL = nba[nba.cluster ==1].reset_index() # Assists - Overall
# AST_OVRL.sort_values(by='APG', ascending=False)
# AST_OVRL

In [52]:
DFP_OVRL = nba[nba.cluster ==3].reset_index() # Defensive Players - Overall
# DFP_OVRL.sort_values(by='RPG', ascending=False)
# DFP_OVRL

In [53]:
AVGP_OVRL = nba[nba.cluster ==0].reset_index() # Average Players - Overall
# AVGP_OVRL.sort_values(by='RPG', ascending=False)
# AVGP_OVRL

## PICKLE

In [54]:
pickle.dump(WKP_OVRL, open('wkp.p', 'wb'))
pickle.dump(BP_OVRL, open('bp.p', 'wb'))
pickle.dump(AST_OVRL, open('ast.p', 'wb'))
pickle.dump(DFP_OVRL, open('dfp.p', 'wb'))
pickle.dump(AVGP_OVRL, open('avgp.p', 'wb'))

## Randomized Players - Clusters

In [14]:
# Weaker_Players = random.sample(list(WKP_OVRL['Player']), k=5)
# Weaker_Players

In [15]:
# Best_Players = random.sample(list(BP_OVRL['Player']), k=5)
# Best_Players

In [16]:
# Assist_Players = random.sample(list(AST_OVRL['Player']), k=5)
# Assist_Players

In [17]:
# Defensive_Players = random.sample(list(DFP_OVRL['Player']), k=5)
# Defensive_Players

In [18]:
# Average_Players = random.sample(list(AVGP_OVRL['Player']), k=5)
# Average_Players

## Players Tuple

In [19]:
# def convert(list):
#     return tuple(list)

In [20]:
# P1 = convert(Weaker_Players)

In [21]:
# P2 = convert(Best_Players)

In [22]:
# P3 = convert(Assist_Players)

In [23]:
# P4 = convert(Defensive_Players)

In [24]:
# P5 = convert(Average_Players)

## Reset Index

In [46]:
nba = nba.groupby(['Player']).mean().reset_index()
nba.head()

,Player,MPG,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,...,PF,ORB,DRB,RPG,APG,SPG,BPG,PPG,Season,cluster
0,Aaron Gordon,30.733333,5.100000,10.933333,0.471667,1.133333,3.433333,0.3320,2.133333,3.166667,...,1.966667,1.700000,4.70,6.466667,3.033333,0.70,0.666667,13.533333,2020.0,1
1,Aaron Holiday,19.933333,2.833333,6.833333,0.415333,0.933333,2.533333,0.3710,0.866667,1.100000,...,1.600000,0.233333,1.50,1.766667,2.433333,0.70,0.233333,7.500000,2020.0,0
2,Aaron Nesmith,12.250000,1.400000,3.550000,0.396000,0.750000,2.250000,0.3185,0.350000,0.400000,...,1.650000,0.450000,1.70,2.150000,0.450000,0.25,0.100000,3.900000,2020.5,2
3,Abdel Nader,13.100000,1.550000,3.650000,0.405500,0.600000,1.650000,0.3305,0.650000,0.950000,...,1.400000,0.300000,1.60,1.850000,0.600000,0.50,0.350000,4.350000,2020.0,2
4,Al Horford,29.700000,4.700000,10.300000,0.455000,1.400000,4.300000,0.3270,1.450000,1.750000,...,2.200000,1.550000,5.95,7.500000,3.700000,0.80,1.300000,12.200000,2020.0,1


## Game

In [47]:
my_team_name = input('name your team: ')
opp_team_name = input('name your team: ')
print('your match is against {}'.format(opp_team_name))
pass_text = [' passes it to ', ' puts it in the path of ']
defend_text = [' performs a great block ' , ' stole the ball ', ' huge defensive rebound ']
shoot_text = [ 'hits the ball ' , ' curls it towards the goal ' , ' shoots ']
goal = [' It went IN!', 'Perfect Shoot!']
no_goal = " he missed it!"

# opp_team = ['jamie' , 'kurt' , 'andy' , 'jack' , 'sam' , 'michael' , 'roberts' , 'samuel' , 'zack' , 'charlie' , ' bob']

def matchStart() :
    my_team_score = 0
    opp_team_score = 0
    match_time = 0
    print("ref blows the whistle and we're under way!")
    while match_time < 10 :
        goal_or_not = random.randint(0,1)
        whose_ball = random.randint(0,1)
        if whose_ball == 0 :
            time.sleep(2)
            print("the ball is taken and {} {} {} {} ".format(random.choice(opp_team),random.choice(pass_text),random.choice(opp_team),random.choice(shoot_text)))
            time.sleep(2)
            if goal_or_not == 1 :
                print("{} score!".format(opp_team_name))
                opp_team_score += 2
                time.sleep(2)
                print(" it's {} {} ".format(str(my_team_score) , str(opp_team_score)))
                match_time += 1
            else :
                time.sleep(2)
                print("{} {} {} {}".format(random.choice(my_team),random.choice(pass_text),random.choice(my_team),random.choice(shoot_text)))
                time.sleep(2)
                print("{} {}!".format(opp_team_name,no_goal))
                match_time += 1
        else :
            if goal_or_not == 1 :
                    time.sleep(2)
                    print("{} {} {} {}".format(random.choice(my_team),random.choice(pass_text),random.choice(my_team),random.choice(shoot_text)))
                    time.sleep(2)
                    print("{} {}!".format(my_team_name,goal) )
                    my_team_score += 2
                    time.sleep(2)
                    print(" it's {} {} ".format(str(my_team_score) , str(opp_team_score)))
                    match_time += 1
            else :
                    time.sleep(2)
                    print("{} {} {} {}".format(random.choice(my_team),random.choice(pass_text),random.choice(my_team),random.choice(shoot_text)))
                    time.sleep(2)
                    print("{} {}!".format(my_team_name,no_goal))
                    time.sleep(2)
                    match_time += 1

    if my_team_score > opp_team_score :
        print("{} {} {} win!".format(my_team_score,opp_team_score,my_team_name))
    elif my_team_score < opp_team_score :
        print("{} {} {} win!".format(my_team_score,opp_team_score,opp_team_name))
    else :
        print("{} {} It's a tie".format(my_team_score,opp_team_score))




# manager_name = input('enter name of manager: ')
# print('hi {} , you have $500 million dollars to start with and build your very own dream team .'\
# "you can buy 5 players.listed player prices are in million ".format(manager_name))

# my_budget = 500
# players = {'hart': 20 ,'curtois': 18 ,'navas': 20 ,'bravo': 15,'cech': 15 ,'de gea': 20 , 'ramos': 40 , 'pique': 30 , 'ronaldo': 50 ,
# 'messi': 50 , 'suarez': 45 , 'neymar': 48 , 'bale': 45 , 'hazard': 42 , 'aguero': 40 , 'smith': 2 , 'johhny': 4 , 'sunny': 8 , 'patrick': 5,
# 'debian': 3 , 'shaggy': 4.5 , 'randy': 3 , 'karim': 6 , 'samuel':2 , 'billy':4 , "aaron":5 }

###########################################################################################################################

                                                # Player1 #

###########################################################################################################################

                                                # Player Randomizer #

Weaker_Players = random.sample(list(WKP_OVRL['Player']), k=5)
Best_Players = random.sample(list(BP_OVRL['Player']), k=5)
Assist_Players = random.sample(list(AST_OVRL['Player']), k=5)
Defensive_Players = random.sample(list(DFP_OVRL['Player']), k=5)
Average_Players = random.sample(list(AVGP_OVRL['Player']), k=5)

###########################################################################################################################

                                                # Tuple Transformation #

def convert(list):
    return tuple(list)

    
P1 = convert(Weaker_Players)
P2 = convert(Best_Players)
P3 = convert(Assist_Players)
P4 = convert(Defensive_Players)
P5 = convert(Average_Players)

###########################################################################################################################

                                                # First Person #

# players=dict()
# for element in c5:
#     players[element] = 5
    
# print(players)

p1=dict()
for element in P1:
    p1[element] = 5
    
p2=dict()
for element in P2:
    p2[element] = 250
    
p3=dict()
for element in P3:
    p3[element] = 100
    
p4=dict()
for element in P4:
    p4[element] = 100
    
p5=dict()
for element in P5:
    p5[element] = 45
    
    
new={"Player1":p1,
     "Player2":p2,
     "Player3":p3,
     "Player4":p4,
     "Player5":p5}


my_team=[]
for cluster in new:
    print(new[cluster])
    user = input("Choose the players: ")
    for element in new[cluster]:
        
        if user==element:
            my_team.append(element)
            
###########################################################################################################################

print('\n')
print('Second player you can choose your team now!')
print('\n')

###########################################################################################################################

                                                # Player2 #
    
###########################################################################################################################

                                                # Player Randomizer #

Weaker_Players = random.sample(list(WKP_OVRL['Player']), k=5)
Best_Players = random.sample(list(BP_OVRL['Player']), k=5)
Assist_Players = random.sample(list(AST_OVRL['Player']), k=5)
Defensive_Players = random.sample(list(DFP_OVRL['Player']), k=5)
Average_Players = random.sample(list(AVGP_OVRL['Player']), k=5)

###########################################################################################################################

                                                # Tuple Transformation #

def convert(list):
    return tuple(list)    


P1 = convert(Weaker_Players)
P2 = convert(Best_Players)
P3 = convert(Assist_Players)
P4 = convert(Defensive_Players)
P5 = convert(Average_Players)
      
###########################################################################################################################

                                                # Second Person #

p1=dict()
for element in P1:
    p1[element] = 5
    
p2=dict()
for element in P2:
    p2[element] = 250
    
p3=dict()
for element in P3:
    p3[element] = 100
    
p4=dict()
for element in P4:
    p4[element] = 100
    
p5=dict()
for element in P5:
    p5[element] = 45
    
    
new={"Player1":p1,
     "Player2":p2,
     "Player3":p3,
     "Player4":p4,
     "Player5":p5}


opp_team =[]
for cluster in new:
    print(new[cluster])
    user = input("Choose the players: ")
    for element in new[cluster]:
        
        if user==element:
            opp_team.append(element)

###########################################################################################################################

# my_team = []

# for key,value in players.items():
#     print("Name: {} Value: {} Million Dollars".format(key,value))

# while len(my_team) < 5:
#     buy_player = input('name player to buy: ')
#     if buy_player in my_team:
#         print(buy_player + " is already in your team")
#         continue
#     if buy_player not in players:
#         print('that player is not available for transfer . check the transfer list again')
#         continue
#     print("Player: {} Value: {} Million Dollars; {}'s Budget: {} Million Dollars".format(buy_player, players[buy_player], manager_name, my_budget))
#     ask_final = input('are you sure you want to make this purchase ? (yes or no) , this cannot be reversed; ')
#     if ask_final.lower() != "yes":
#         continue
#     if my_budget - players[buy_player] <= 0:
#         print("insufficient funds")
#         ask_to_sell = input('you need to sell players , enter name of player to sell')
#         if ask_to_sell not in my_team :
#             continue
#         print("{} ,  Value: {} Million Dollars.Sold ".format(ask_to_sell,players[buy_player]))
#         my_team.remove(ask_to_sell)
#         my_budget += players[ask_to_sell]
#         print("{} is your new budget".format(my_budget))
#         print("you have {} players in your team".format(str(len(my_team))))
#         continue
#     my_budget -= players[buy_player]
#     my_team.append(buy_player)
#     print("you have {} players in your team".format(str(len(my_team))))

# print("\n".join(my_team))
# print('  This is your final team.May you achieve victory and glory with them!')


matchStart()

name your team: A
name your team: B
your match is against B
{'Bruno Fernando': 5, 'Moe Wagner': 5, 'Vince Carter': 5, 'Abdel Nader': 5, 'Obi Toppin': 5}
Choose the players: Abdel Nader
{'Domantas Sabonis': 250, 'Karl-Anthony Towns': 250, 'Cole Anthony': 250, 'Joel Embiid': 250, 'Kyrie Irving': 250}
Choose the players: Obi Toppin
{'Jonas Valanciunas': 100, 'Will Barton': 100, 'Gary Trent, Jr.': 100, 'Kevin Huerter': 100, 'T.J. Warren': 100}
Choose the players: Will Barton
{'Pat Connaughton': 100, 'Patrick Williams': 100, 'Robert Covington': 100, 'Shabazz Napier': 100, 'Immanuel Quickley': 100}
Choose the players: Patrick Williams
{'Solomon Hill': 45, 'Kevin Knox': 45, 'Andre Iguodala': 45, 'Jeff Teague': 45, 'Torrey Craig': 45}
Choose the players: Andre Iguodala


Second player you can choose your team now!


{'Elie Okobo': 5, 'Goga Bitadze': 5, 'Vince Carter': 5, 'Rodions Kurucs': 5, 'Moe Wagner': 5}
Choose the players: Goga Bitadze
{'Jayson Tatum': 250, "D'Angelo Russell": 250, 'Nikol

## Testing AREA

In [ ]:
new_Dict= {"culster_5": (,),
          "cluster_4": (,)}

In [ ]:
print("escolhe 1 jogador dos seguintes:", new_dict["cluster_5"])

In [ ]:
new={"cluster5":c5,
     "cluster2":{"ze":5, "ruben":5}}

In [ ]:
new={"cluster5":c5,
     "cluster2":{"ze":5, "ruben":5}}


my_choices=[]
for cluster in new:
    print(new[cluster])
    user = input("escolhe:")
    for element in new[cluster]:
        
        if user==element:
            my_choices.append(element)
        

In [ ]:
my_choices

In [ ]:
c5 = ('Draymond Green', 'Svi Mykhailiuk', 'Mohamed Bamba', 'Donte DiVincenzo', 'Damion Lee', 
                         'T.J. McConnell', 'JaMychal Green', 'Derrick Favors', 'Bruce Brown, Jr.','Isaiah Hartenstein', 
                         'Chris Boucher', 'Jarrett Culver', 'Garrett Temple', 'Elfrid Payton', 'DeAndre Jordan')
# dict_cost=dict()
# for element in Players:
#     dict_cost[element] = 5
    
# dict_cost


p1=dict()
for element in P1:
    p1[element] = 5
    
p2=dict()
for element in P2:
    p2[element] = 250
    
p3=dict()
for element in P3:
    p3[element] = 100
    
p4=dict()
for element in P4:
    p4[element] = 100
    
p5=dict()
for element in P5:
    p5[element] = 45
    
    
new={"Player1":p1,
     "Player2":p2,
     "Player3":p3,
     "Player4":p4,
     "Player5":p5}


players=[]
for cluster in new:
    print(new[cluster])
    user = input("escolhe:")
    for element in new[cluster]:
        
        if user==element:
            players.append(element)



In [ ]:
players

In [ ]:
opp_team =[]
for cluster in new:
    print(new[cluster])
    user = input("Choose the players: ")
    for element in new[cluster]:
        
        if user==element:
            opp_team.append(element)

In [36]:
nba[nba['Player'].isin(my_team)]['FG%'].values.sum()

2.4656666666666665

In [37]:
print(nba[nba['Player'].isin(opp_team)])#['FG%'].values.sum()

2.42

In [ ]:
opp_team

In [56]:
Weaker_Players

['Elie Okobo', 'Goga Bitadze', 'Vince Carter', 'Rodions Kurucs', 'Moe Wagner']